In [16]:
!pip3 install -U python-docx
!pip3 install -U textract
!apt-get install antiword
!pip3 install -U bert-extractive-summarizer
!pip3 install -U transformers

  Using cached python-docx-0.8.10.tar.gz (5.5 MB)
  Created wheel for python-docx: filename=python_docx-0.8.10-py3-none-any.whl size=184489 sha256=926e5e4fd525243a20e0ee67deaffecace72f244ddf6883d5403c45508b7d72b
  Stored in directory: /home/samoed/.cache/pip/wheels/97/4c/2e/68066cbf12b9b2e66403da8982aaf4f656d9f5cb5dc3179e82
Successfully built python-docx
     |████████████████████████████████| 103 kB 1.2 MB/s eta 0:00:01
     |████████████████████████████████| 111 kB 12.9 MB/s eta 0:00:01
     |████████████████████████████████| 45 kB 116 kB/s eta 0:00:011
     |████████████████████████████████| 97 kB 2.1 MB/s  eta 0:00:01
     |████████████████████████████████| 8.9 MB 18 kB/s  eta 0:00:011
     |████████████████████████████████| 5.6 MB 3.6 MB/s eta 0:00:01
  Using cached SpeechRecognition-3.8.1-py2.py3-none-any.whl (32.8 MB)
     |████████████████████████████████| 73 kB 611 kB/s eta 0:00:011
     |████████████████████████████████| 147 kB 11.2 MB/s eta 0:00:01
     |████████████████████

     |████████████████████████████████| 2.1 MB 1.3 MB/s eta 0:00:01
  Attempting uninstall: transformers
    Found existing installation: transformers 4.4.2
    Uninstalling transformers-4.4.2:
      Successfully uninstalled transformers-4.4.2


In [1]:
from peewee import *

database = SqliteDatabase('bd.db')

class BaseModel(Model):
    class Meta:
        database = database

class Campus(BaseModel):
    id = AutoField(column_name='Id', null=True)
    campus_name = TextField(column_name='Campus_name', null=True)

    class Meta:
        table_name = 'Campus'

class Cathedra(BaseModel):
    id = AutoField(column_name='Id', null=True)
    cathedra_name = TextField(column_name='Cathedra_Name', null=True)

    class Meta:
        table_name = 'Cathedra'

class Faculties(BaseModel):
    id = AutoField(column_name='Id', null=True)
    name_faculties = TextField(column_name='Name_Faculties', null=True)

    class Meta:
        table_name = 'Faculties'

class EducationalProgram(BaseModel):
    id = AutoField(column_name='Id', null=True)
    name_educational_program = TextField(column_name='Name_EducationalProgram', null=True)

    class Meta:
        table_name = 'EducationalProgram'

class Position(BaseModel):
    id = AutoField(column_name='Id', null=True)
    position_name = TextField(column_name='PositionName', null=True)

    class Meta:
        table_name = 'Position'

class Professors(BaseModel):
    id = AutoField(column_name='Id', null=True)
    prof_name = TextField(column_name='Prof_Name', null=True)
    profile_link = TextField(column_name='Profile_Link', null=True)
    cathedra = ForeignKeyField(column_name='Cathedra_Id', model=Cathedra, null=True)
    position = ForeignKeyField(column_name='Position_Id', model=Position, null=True)
    competence = TextField(column_name='Competence', null=True)
    embeddings = TextField(column_name='Embeddings', null=True)
    
    class Meta:
        table_name = 'Professors'

class Vkr(BaseModel):
    id = AutoField(column_name='Id', null=True)
    vkr_name = TextField(column_name='VKR_Name', null=True)
    vkr_link = TextField(column_name='VKR_Link', null=True)
    student = TextField(column_name='Student', null=True)
    prof = ForeignKeyField(column_name='Prof_Id', model=Professors, null=True)
    campus = ForeignKeyField(column_name='Campus_Id', model=Campus, null=True)
    educational_program = ForeignKeyField(column_name='EducationalProgram_Id', model=EducationalProgram, null=True)
    vkr_text_link = TextField(column_name='VKRText_Link', null=True)
    

    class Meta:
        table_name = 'VKR'

In [2]:
def CreateTables():
    Campus.create_table()
    Cathedra.create_table()
    Faculties.create_table()
    EducationalProgram.create_table()
    Position.create_table()
    Professors.create_table()
    Vkr.create_table()
    
CreateTables()

In [3]:
import requests
import pandas as pd
from tqdm import tqdm
from selenium import webdriver
from bs4 import BeautifulSoup
import urllib.request
import time
from docx import Document
import textract
import magic

from transformers import BertTokenizer, BertModel, BertConfig
import torch
from collections import Counter
import string
from summarizer import Summarizer

In [4]:
#get links on letters for people
r = requests.get('https://www.hse.ru/org/persons/?udept=135213')
page = BeautifulSoup(r.text, 'html.parser')
url = 'https://www.hse.ru'
letters = []
for link in page.findAll("div", {'class': "abc-filter__letter"}):
    if (str(link.contents)).find('"') != -1:
            s = url + '/org/persons/' + str(link.contents)[10:34]
            letters.append(str(s))
print(len(letters))

25


In [5]:
#get array of people and links to their profile
people=[]
for letter in tqdm(letters):
    r = requests.get(letter)
    page = BeautifulSoup(r.text, 'html.parser')
    url = 'https://www.hse.ru'
    for link in page.findAll("a", {'class':"link"}):
        try:
            if (link['href'][:13]=="/org/persons/" or link['href'][:7]=="/staff/") and link['href'][13]!='?' and link.text != 'полный список':
                people.append({
                    'name': link.text[1:],
                    'link': url + link['href']
                })
        except:
            a=1
df = pd.DataFrame(people)
df.head(10)

100%|██████████| 25/25 [00:42<00:00,  1.68s/it]


,name,link
0,Абашева Александра Сергеевна,https://www.hse.ru/org/persons/485801
1,Авраменко Иван Александрович,https://www.hse.ru/org/persons/99247481
2,Агаркова Наталия Владиславовна,https://www.hse.ru/org/persons/28123133
3,Айхбергер Юрген Томас Германн,https://www.hse.ru/org/persons/359359486
4,Алексеева Лариса Николаевна,https://www.hse.ru/org/persons/66770114
5,Аленина Карина Анатольевна,https://www.hse.ru/org/persons/202287
6,Алова Надежда Владимировна,https://www.hse.ru/org/persons/28123313
7,Алферова Татьяна Викторовна,https://www.hse.ru/org/persons/359862051
8,Андреева Ольга Юрьевна,https://www.hse.ru/org/persons/152893083
9,Андрианов Игорь Владимирович,https://www.hse.ru/org/persons/223725475


In [6]:
#get list of cathedras and positions of people
cathedra = set()
position = set()
for person in tqdm(people):
    r = requests.get(person['link'])
    page = BeautifulSoup(r.text, 'html.parser')
    cathedra.add(page.find("ul", {"class":"g-ul g-list small"}).findAll("a")[-1].text)
    position.add(page.find("span", {"class":"person-appointment-title"}).text[:-1])

100%|██████████| 344/344 [10:44<00:00,  1.87s/it]


In [7]:
#save data
for i in cathedra:
    cathedraAdd = Cathedra(cathedra_name=i)
    cathedraAdd.save()
for i in position:
    positionAdd = Position(position_name=i)
    positionAdd.save()

In [8]:
campus = Campus(campus_name="Пермь")
campus.save()

1

In [10]:
#add proffesors to bd
vkr = []
for person in tqdm(people):
    r = requests.get(person['link'])
    page = BeautifulSoup(r.text, 'html.parser')
    cathedraPerson = page.find("ul", {"class":"g-ul g-list small"}).findAll("a")[-1].text
    positionPerson = page.find("span", {"class":"person-appointment-title"}).text[:-1]
    idCathedra = Cathedra.get(Cathedra.cathedra_name == cathedraPerson)
    idPosition = Position.get(Position.position_name == positionPerson)
    profName = person['name']
    profileLink = person['link']
    
    prof = Professors(
        cathedra = idCathedra,
        competence = "",
        embeddings = "",
        position = idPosition,
        prof_name = profName,
        profile_link = profileLink
    )
    prof.save()
    
    for link in page.findAll("a", {'class':"link"}):
            if link['href'][:9]=='/edu/vkr/':
                    #print(url+link['href'])
                    vkr.append(url+link['href'])

100%|██████████| 344/344 [09:17<00:00,  1.62s/it]


In [11]:
#get list of OP and facultets
browser = webdriver.Firefox(executable_path = '../.browserDrivers/geckodriver')
OP = set()
facult = set()
for link in tqdm(vkr):
    browser.get(link)
    browser.implicitly_wait(1)
    time.sleep(1)
    page = BeautifulSoup(browser.page_source, 'html.parser')
    for elem in page.findAll("p",{'class':'vkr-card__item'}):
        elem_text = elem.text.split(':')[1].strip()
        elem_name = elem.text.split(':')[0].strip()
        #print(elem_name)
        #print(elem_text)
        if elem_name =='Кампус/факультет':
            #print(elem_name)
            facult.add(elem_text)
        elif elem_name =='Программа':
            #print(elem_name)
            OP.add(elem_text)

100%|██████████| 1790/1790 [51:17<00:00,  1.72s/it] 


In [12]:
#save data
for i in facult:
    facultAdd = Faculties(name_faculties=i)
    facultAdd.save()
    
for i in OP:
    facultAdd = EducationalProgram(name_educational_program=i)
    facultAdd.save()

In [13]:
#get list of vkrs
browser = webdriver.Firefox(executable_path = '../.browserDrivers/geckodriver')
vkr_data = []
for link in tqdm(vkr):
    #print(link)
    browser.get(link)
    browser.implicitly_wait(1)
    time.sleep(1)
    page = BeautifulSoup(browser.page_source, 'html.parser')
    vkr_tmp = []
    name_vkr=''
    name_student=''
    name_prof=''
    campus=''
    program=''
    grade=-1
    year=-1
    link_file=''
    for elem in page.findAll("h1"):
        #print(elem.text)
        name_vkr=elem.text
    for elem in page.findAll("p",{'class':'vkr-card__item'}):
        #print(elem.text)
        elem_name = elem.text.split(':')[0].strip()
        elem_text = elem.text.split(':')[1].strip()
        if elem_name =='ФИО студента':
            name_student = elem_text
        elif elem_name =='Руководитель':
            name_prof = elem_text
        elif elem_name =='Кампус/факультет':
            facultElem = elem_text
        elif elem_name =='Программа':
            program = elem_text
        elif elem_name =='Год защиты':
            year = int(elem_text)
        elif elem_name =='Оценка':
            grade = int(elem_text)
            
    for elem in page.findAll("a", {'class':'vkr-icon__text link link_no-underline'}):
            link_file=elem['href']
    vkrElem = Vkr(
        facult = Faculties.get(Faculties.name_faculties == facultElem),
        educational_program = EducationalProgram.get(EducationalProgram.name_educational_program == program),
        prof = Professors.get(Professors.prof_name == name_prof),
        student = name_student,
        vkr_text_link = link_file,
        vkr_link = link,
        vkr_name = name_vkr,
        campus = 1
    )    
    vkrElem.save()

100%|██████████| 1790/1790 [1:12:25<00:00,  2.43s/it]


In [78]:
textLinks = Vkr.select().where(Vkr.vkr_text_link != "").join(Professors, on=(Vkr.prof == Professors.id))
for i in textLinks:
    print(i.vkr_text_link)
len(textLinks)
print(textLinks[4].vkr_text_link)

https://lms.hse.ru/ap_service.php?getwork=1&guid=7FBCBD17-2BD2-4324-9320-DA9AB18846B7
https://www.hse.ru/data/2014/05/19/1322118389/Диссертация_19Мая_отправка.docx
https://www.hse.ru/data/2014/05/21/1321758023/Пугачевский А.В. Разработка коммерческог.. Кондитерская фабрика Пермская.docx
https://www.hse.ru/data/2013/08/21/1289969400/Тужанская Елена.doc
https://lms.hse.ru/ap_service.php?getwork=1&guid=7F05C602-D501-405F-ABA5-CD3D3DFE300D
https://lms.hse.ru/ap_service.php?getwork=1&guid=35592BC0-9CE0-4F71-B877-D31B060DE9F5
https://lms.hse.ru/ap_service.php?getwork=1&guid=7F05C602-D501-405F-ABA5-CD3D3DFE300D


In [74]:
df["name"]==textLinks[4].prof.prof_name

0      False
1      False
2      False
3      False
4      False
       ...  
339    False
340    False
341    False
342    False
343    False
Name: name, Length: 344, dtype: bool

In [23]:
mime = magic.Magic(mime=True)
textLinks = Vkr.select().where(Vkr.vkr_text_link != "").join(Professors, on=(Vkr.prof == Professors.id))
df['fullText'] = ""
unsuccessful = []
successful = []
docx = "application/vnd.openxmlformats-officedocument.wordprocessingml.document"
pdf = "application/pdf"
doc = "application/msword"
for row in tqdm(textLinks):
    url = row.vkr_text_link
    r = requests.get(url, allow_redirects=True)
    open('file', 'wb').write(r.content)
    fileType = mime.from_file("file")
    if fileType == docx:
        open('file.docx', 'wb').write(r.content)
        f = open('file.docx', 'rb')
        document = Document(f)
        f.close()
        fullText = ""
        for para in document.paragraphs:
            fullText += (para.text)+" "
        successful.append(url)
    elif fileType == doc:
        open('file.doc', 'wb').write(r.content)
        fullText = textract.process("file.doc").decode("utf-8")
        successful.append(url)
    elif fileType == pdf:
        open('file.pdf', 'wb').write(r.content)
        try:
            fullText = textract.process("file.pdf").decode("utf-8")
        except TypeError:
            unsuccessful.append(url)
            continue
        successful.append(url)
    else:
        unsuccessful.append(url)
        continue
    df.loc[df["name"]==row.prof.prof_name, 'fullText' ] += fullText

  0%|          | 1/621 [00:01<13:21,  1.29s/it]

application/msword


  0%|          | 2/621 [00:02<12:42,  1.23s/it]

application/vnd.openxmlformats-officedocument.wordprocessingml.document


  0%|          | 3/621 [00:03<13:28,  1.31s/it]

...

application/vnd.openxmlformats-officedocument.wordprocessingml.document


 36%|███▌      | 224/621 [10:59<20:05,  3.04s/it]

application/vnd.openxmlformats-officedocument.wordprocessingml.document


 36%|███▌      | 224/621 [11:11<19:49,  3.00s/it]


KeyboardInterrupt: 

In [ ]:
df.to_csv("fullText.csv")

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

custom_config = BertConfig.from_pretrained('DeepPavlov/rubert-base-cased')
custom_config.output_hidden_states=True
custom_tokenizer = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')
custom_model = BertModel.from_pretrained('DeepPavlov/rubert-base-cased', config=custom_config)
custom_model.eval()
custom_model.to('cuda')

In [ ]:
%%time
modelSum = Summarizer(custom_model=custom_model, custom_tokenizer=custom_tokenizer)
df['shortText'] = ""
df['embeddings'] = ""
for row in tqdm(df.index):
    text = df.loc[row, 'fullText'][:1000000]
    result = modelSum(text, num_sentences=10)
    embeddings = modelSum.run_embeddings(text, num_sentences=10)
    summary = "".join(result)
    df.loc[row, 'shortText'] = summary